## Application des transformations definies dans "trasnfromers.py"

In [1]:
from transformers import *

In [2]:
import pandas as pd
from abc import ABC, abstractmethod
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from pathlib import Path
from sklearn.model_selection import train_test_split

import pickle


In [3]:
columns_to_drop = [
        "piezo_station_department_name",
        "piezo_station_update_date",
        "piezo_station_commune_code_insee",
        "piezo_station_pe_label",
        "piezo_station_bdlisa_codes",
        "piezo_station_bss_code",
        "piezo_station_bss_id", 
        "piezo_bss_code",
        "piezo_measurement_date",
        "piezo_producer_name",
        "piezo_measure_nature_code",
        "meteo_name",
        "meteo_id", 
        "meteo_latitude",
        "meteo_longitude",
        "hydro_station_code",
        "hydro_method_code", 
        "hydro_method_label", 
        "hydro_qualification_label", 
        "prelev_structure_code_0",
        "prelev_structure_code_2",
        "prelev_structure_code_0",
        "prelev_commune_code_insee_0",
        "piezo_station_department_code",
        
        "meteo_DRR", 
        "meteo_temperature_min_ground", 
        "meteo_temperature_min_50cm", 
        "meteo_pressure_avg",
        "meteo_pression_maxi", 
        "meteo_wind_speed_avg_2m", 
        "meteo_wind_max_2m", 
        "meteo_wind_direction_max_inst_2m", 
        "meteo_time_wind_max_2m", 
        "meteo_wetting_duration", 
        "meteo_sunshine_duration", 
        "meteo_radiation", 
        "meteo_radiation_direct", 
        "meteo_sunshine_%", 
        "meteo_radiation_IR", 
        "meteo_radiation_UV_max", 
        "meteo_cloudiness", 
        "meteo_cloudiness_height", 
        "meteo_if_snow", 
        "meteo_if_fog", 
        "meteo_if_thunderstorm", 
        "meteo_if_sleet", 
        "meteo_if_hail", 
        "meteo_if_dew", 
        "meteo_if_black_ice", 
        "meteo_if_snow_ground", 
        "meteo_if_frost", 
        "meteo_if_smoke", 
        "meteo_if_mist",
        "meteo_if_lightning", 
        "meteo_evapotranspiration_Monteith", 
        "meteo_radiation_UV", 
        "meteo_snow_height", 
        "meteo_snow_thickness_max", 
        "meteo_snow_thickness_6h"]

ajouts_drop_yael = ["meteo_altitude","meteo_temperature_min","meteo_temperature_max"]


columns_to_drop+=ajouts_drop_yael

In [4]:
# Specification des colonnes  a garder 

##ANCIENNE VERSION
#all_columns = pd.read_csv(path_src_dataset, nrows=0).columns.tolist()
#columns_to_keep = [col for col in all_columns if col not in columns_to_drop] 


##NOUVELLE VERSION 
#Je précise ci dessous les colonnes nécessaires pour chacun des transformers, et en commentaire l'ordre dans lequel ce transformer sera utilisé
#Date
col_yass = ['meteo_date'] #ORDRE = 4 (en dernier car il supprime les dates )
#Altitude
col_flo = ["piezo_station_altitude", "meteo_altitude"] # ORDRE 1
#Insee & rain "CleanFeatures"
cols_yael_input = ['insee_%_agri', 'meteo_rain_height', 'insee_pop_commune', 'insee_med_living_level', 'insee_%_ind', 'insee_%_const']
cols_yael_need = ["piezo_station_department_code", "meteo_date"]

#Temperature
cols_lucien_need = ['piezo_station_department_code', 'piezo_measurement_date']
cols_lucien_input = ['meteo_temperature_avg','meteo_temperature_min','meteo__pressure_saturation_avg','meteo_temperature_max']
#Lat Long
cols_mat = ["distance_piezo_meteo",'piezo_station_longitude','piezo_station_latitude','meteo_latitude','meteo_longitude']

#target
target = "piezo_groundwater_level_category"

columns_to_keep = col_yass + cols_yael_input + cols_yael_need + col_flo + cols_lucien_need + cols_lucien_input + cols_mat + [target]

columns_to_keep

['meteo_date',
 'insee_%_agri',
 'meteo_rain_height',
 'insee_pop_commune',
 'insee_med_living_level',
 'insee_%_ind',
 'insee_%_const',
 'piezo_station_department_code',
 'meteo_date',
 'piezo_station_altitude',
 'meteo_altitude',
 'piezo_station_department_code',
 'piezo_measurement_date',
 'meteo_temperature_avg',
 'meteo_temperature_min',
 'meteo__pressure_saturation_avg',
 'meteo_temperature_max',
 'distance_piezo_meteo',
 'piezo_station_longitude',
 'piezo_station_latitude',
 'meteo_latitude',
 'meteo_longitude',
 'piezo_groundwater_level_category']

In [5]:
path_src_dataset = Path("./data/src/X_train_Hi5.csv")

out_folder_dataset = Path("./data/cleaned")
# Create the folder if it doesn't exist
out_folder_dataset.mkdir(parents=True, exist_ok=True)

out_folder_config = Path("./data/processed/pipelines")
out_folder_config.mkdir(parents=True, exist_ok=True)

# Load the CSV file with only the relevant columns
df = pd.read_csv(path_src_dataset, usecols=columns_to_keep,nrows=10e4) #  SI on veut charger moins de lignes : ajouter --> ,nrows=10e4)

target = "piezo_groundwater_level_category"
X = df.drop(columns=target)

#Mapping du target 
mapping = {'Very Low': 0, 'Low': 1, 'Average': 2, 'High': 3, 'Very High': 4}
y = df[target].map(mapping)

#Test-val split 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Apply the transformers selected
processing_pipeline = Pipeline(steps=[
("DropNaRate", DropNaRate(0.7)),
("CleanFeatures", CleanFeatures(cols_yael_input)),
("Altitude", AltitudeTrans(columns=["piezo_station_altitude", "meteo_altitude"])),
('LatLong',CleanLatLon()),
('Temp',TemperaturePressionTrans(columns=cols_lucien_input)),
('Dates',DateTransformer()),
('DropCols',DropCols(columns_to_drop) )

# ... Add others transformations
])

In [6]:
print("Pipeline ongoing...")
processed_X_train = processing_pipeline.fit_transform(X_train)
processed_X_val = processing_pipeline.transform(X_val)

Pipeline ongoing...
>> (Info) Droped columns : []
>> (Info) Recuperations des moyennes des données INSEE par department
>> (Info) Infos medianes Insee recupérees
>> (Info) Valeurs Manquantes comblées avec les Médianes.
>> (INFO - DropCols) columns ['piezo_station_department_name', 'piezo_station_update_date', 'piezo_station_commune_code_insee', 'piezo_station_pe_label', 'piezo_station_bdlisa_codes', 'piezo_station_bss_code', 'piezo_station_bss_id', 'piezo_bss_code', 'piezo_measurement_date', 'piezo_producer_name', 'piezo_measure_nature_code', 'meteo_name', 'meteo_id', 'meteo_latitude', 'meteo_longitude', 'hydro_station_code', 'hydro_method_code', 'hydro_method_label', 'hydro_qualification_label', 'prelev_structure_code_0', 'prelev_structure_code_2', 'prelev_structure_code_0', 'prelev_commune_code_insee_0', 'piezo_station_department_code', 'meteo_DRR', 'meteo_temperature_min_ground', 'meteo_temperature_min_50cm', 'meteo_pressure_avg', 'meteo_pression_maxi', 'meteo_wind_speed_avg_2m', 'm

In [ ]:
processed_X_train.describe()

,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,meteo_date,meteo_rain_height,meteo_temperature_avg,meteo__pressure_saturation_avg,distance_piezo_meteo,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,149.880498,2.221560,46.422956,0.924389,1.925792,37.687337,75.091392,0.950250,9.642474,7003.475497,22963.870947,13.193301,11.307937
std,180.345925,2.724138,2.174254,0.067875,4.578783,57.633061,68.844381,0.217429,15.136276,32403.892507,2921.272189,16.728652,12.585763
min,0.720000,-4.657129,41.399732,0.772456,0.000000,-8.300000,0.500000,0.000000,0.000000,6.000000,146.429890,0.000000,0.000000
25%,41.900000,-0.105584,44.499602,0.869764,0.000000,5.600000,8.400000,1.000000,0.500000,482.000000,21130.000000,0.600000,3.200000
50%,108.000000,1.897576,46.393226,0.941397,0.200000,8.700000,13.200000,1.000000,3.400000,1229.000000,22420.000000,7.300000,7.300000
75%,185.000000,4.216516,48.214196,0.985240,2.000000,13.000000,146.429890,1.000000,11.600000,3752.000000,24440.000000,18.800000,15.000000
max,2150.000000,9.519890,50.971087,1.000000,189.000000,146.429890,146.429890,1.000000,146.429890,873076.000000,40950.000000,146.429890,146.429890


In [8]:
stop

NameError: name 'stop' is not defined

In [ ]:

# # Save the processed data to CSV
# processed_X_train.to_csv(out_folder_dataset / "X_train.csv", index=False)
# processed_X_val.to_csv(out_folder_dataset / "X_val.csv", index=False)
# y_train.to_csv(out_folder_dataset / "y_train.csv", index=False)
# y_val.to_csv(out_folder_dataset / "y_val.csv", index=False)

print("Data converted to csv")

Data converted to csv


In [ ]:
#quick KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score


# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=25)  # Default is 5 neighbors
knn.fit(processed_X_train, y_train)

# Make predictions
y_val_pred = knn.predict(processed_X_val)

# Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))


Accuracy: 0.67635

Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.72      0.71      1934
           1       0.66      0.63      0.64      3222
           2       0.64      0.66      0.65      4630
           3       0.65      0.64      0.64      5415
           4       0.74      0.75      0.74      4799

    accuracy                           0.68     20000
   macro avg       0.68      0.68      0.68     20000
weighted avg       0.68      0.68      0.68     20000



In [ ]:
#quick KNN
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

global_pipeline = Pipeline([
    ('processing', processing_pipeline),
    ('StandardScaler', PartialStandardScaler(columns='all')),
    ('estimator', KNeighborsClassifier(n_neighbors=25))
])

In [ ]:
global_pipeline.fit(X_train, y_train)

>> (Info) Droped columns : []
>> (Info) Column insee_%_agri has been standardized to numeric.
>> (Info) Missing values in insee_%_agri filled with median: 3.4
>> (Info) Missing values in meteo_rain_height filled with median: 0.2
>> (INFO - DropCols) columns ['piezo_station_department_name', 'piezo_station_update_date', 'piezo_station_commune_code_insee', 'piezo_station_pe_label', 'piezo_station_bdlisa_codes', 'piezo_station_bss_code', 'piezo_station_bss_id', 'piezo_bss_code', 'piezo_measurement_date', 'piezo_producer_name', 'piezo_measure_nature_code', 'meteo_name', 'meteo_id', 'meteo_latitude', 'meteo_longitude', 'hydro_station_code', 'hydro_method_code', 'hydro_method_label', 'hydro_qualification_label', 'prelev_structure_code_0', 'prelev_structure_code_2', 'prelev_structure_code_0', 'prelev_commune_code_insee_0', 'piezo_station_department_code', 'meteo_DRR', 'meteo_temperature_min_ground', 'meteo_temperature_min_50cm', 'meteo_pressure_avg', 'meteo_pression_maxi', 'meteo_wind_speed_a

AttributeError: 'CleanLatLon' object has no attribute 'dist_to_meteo_threshold'

AttributeError: 'CleanLatLon' object has no attribute 'dist_to_meteo_threshold'

AttributeError: 'CleanLatLon' object has no attribute 'dist_to_meteo_threshold'

In [ ]:
y_val_pred = global_pipeline.predict(X_val)

# Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))

>> (Info) Missing values in insee_%_agri filled with median: 3.4
>> (Info) Missing values in meteo_rain_height filled with median: 0.2
>> (INFO - DropCols) columns ['piezo_station_department_name', 'piezo_station_update_date', 'piezo_station_commune_code_insee', 'piezo_station_pe_label', 'piezo_station_bdlisa_codes', 'piezo_station_bss_code', 'piezo_station_bss_id', 'piezo_bss_code', 'piezo_measurement_date', 'piezo_producer_name', 'piezo_measure_nature_code', 'meteo_name', 'meteo_id', 'meteo_latitude', 'meteo_longitude', 'hydro_station_code', 'hydro_method_code', 'hydro_method_label', 'hydro_qualification_label', 'prelev_structure_code_0', 'prelev_structure_code_2', 'prelev_structure_code_0', 'prelev_commune_code_insee_0', 'piezo_station_department_code', 'meteo_DRR', 'meteo_temperature_min_ground', 'meteo_temperature_min_50cm', 'meteo_pressure_avg', 'meteo_pression_maxi', 'meteo_wind_speed_avg_2m', 'meteo_wind_max_2m', 'meteo_wind_direction_max_inst_2m', 'meteo_time_wind_max_2m', 'me

In [ ]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))

Accuracy: 0.5351

Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.48      0.52      1934
           1       0.52      0.45      0.48      3222
           2       0.51      0.50      0.50      4630
           3       0.51      0.58      0.55      5415
           4       0.58      0.60      0.59      4799

    accuracy                           0.54     20000
   macro avg       0.54      0.52      0.53     20000
weighted avg       0.54      0.54      0.53     20000



### Save Pipeline

In [ ]:
pipeline_name = "1st_pipeline_13h00"

# Writing to sample.json
with open(out_folder_config / Path(pipeline_name + ".pkl"), "wb") as file:
    pickle.dump(global_pipeline, file)